In [ ]:
#Começa aqui após ter os arquivos
import pandas as pd
import geopandas as gpd
import folium
from shapely.wkt import loads
from shapely import wkt
from branca.colormap import linear

In [ ]:
#Shapefile baixado em: https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/15774-malhas.html
'''
file_names = ["2023-Q1.parquet", "2023-Q2.parquet", "2023-Q3.parquet", "2023-Q4.parquet"]
def load_and_process_data(file_names):
    dfs = []
    for file_name in file_names:
        print(f'Carregando dados do arquivo: {file_name}')
        df = pd.read_parquet(file_name)
        df['geometry'] = df['tile'].apply(wkt.loads)
        df['quarter'] = file_name.split('-')[1]  # Assumindo que o nome do arquivo contém o trimestre
        dfs.append(df)
    
    combined_df = pd.concat(dfs, ignore_index=True, axis=0)
    return combined_df

df = load_and_process_data(file_names)
print(df)
Ficou muito pesado
'''
file_name = "2024-Q01.parquet"
df = pd.read_parquet(file_name)


df.rename(columns={'tile': 'geometry'}, inplace=True)

df['geometry'] = df['geometry'].apply(wkt.loads)

df_geo = gpd.GeoDataFrame(df, geometry='geometry')
df_geo.crs = "EPSG:4326"

df_geo.head()

In [ ]:
#shapefile_sp = gpd.read_file('SP_Municipios_2022.zip') 
shapefile_sp = gpd.read_file('BR_Municipios_2022.zip') #Tentar rodar com brasil
distritos = shapefile_sp.to_crs(4326)
distritos.head()

#nome_arquivo_excel = "distritos.xlsx"
#distritos.to_excel(nome_arquivo_excel, index=False)
#print(f"Arquivo '{nome_arquivo_excel}' salvo com sucesso!")

In [ ]:
# Realiza o join espacial
join_sp = gpd.sjoin(df_geo, distritos, how="inner", op='intersects')
join_sp.head()
# O resultado 'join_sp' conterá as velocidades de internet com a informação dos municípios de São Paulo

In [ ]:
# Supondo que join_sp é o DataFrame com seus dados
# Calcular a média de avg_d_kbps por município
media_velocidade_sp = join_sp.groupby('NM_MUN')['avg_d_kbps'].mean()

# Resetar o índice para transformar em DataFrame
media_velocidade_sp = media_velocidade_sp.reset_index()

# Converter a média de kbps para Mbps diretamente na nova coluna
media_velocidade_sp['avg_d_mbps'] = media_velocidade_sp['avg_d_kbps'] / 1000

# Nome do arquivo Excel
#nome_arquivo_excel = "media_velocidade_por_municipio.xlsx"
#media_velocidade_sp.to_excel(nome_arquivo_excel, index=False)

In [ ]:
# Merge para associar cada município à sua média de velocidade de internet
map_data = distritos.merge(media_velocidade_sp, left_on='NM_MUN', right_on='NM_MUN')

# Exibir dataframe resultante
print(map_data)
#map_data.to_excel('teste.xlsx')

In [ ]:
gdf = gpd.GeoDataFrame(map_data, geometry='geometry')
gdf = gdf.set_geometry('geometry')
# Agrupar por estado (SIGLA_UF) e realizar as agregações
grouped = gdf.dissolve(by='SIGLA_UF', aggfunc={
    'AREA_KM2': 'sum',
    'avg_d_kbps': 'mean',
    'avg_d_mbps': 'mean'
})
# Resetar o índice para usar SIGLA_UF como coluna
grouped = grouped.reset_index()
# Mostrar os dados agregados
print(grouped.head())

# Opcionalmente, salvar em um novo arquivo
#grouped.to_file('dados_agregados_estado.geojson', driver='GeoJSON')


In [ ]:
# Criar um mapa base
m = folium.Map(location=[-15.788497, -47.879873], zoom_start=4)

# Definir a paleta de cores
colormap = linear.YlOrRd_09.scale(grouped['avg_d_mbps'].min(), grouped['avg_d_mbps'].max())
colormap.caption = 'Velocidade Média de Internet (Mbps)'

# Adicionar os polígonos dos estados ao mapa
choropleth = folium.Choropleth(
    geo_data=grouped,
    name='choropleth',
    data=grouped,
    columns=['SIGLA_UF', 'avg_d_mbps'],
    key_on='feature.properties.SIGLA_UF',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Velocidade Média de Internet (Mbps)'
).add_to(m)

# Adicionar popups para cada estado
for _, row in grouped.iterrows():
    folium.GeoJson(
        row['geometry'],
        name=row['SIGLA_UF'],
        tooltip=folium.Tooltip(f"{row['SIGLA_UF']}: {row['avg_d_mbps']} Mbps")
    ).add_to(m)

# Adicionar controle de camadas
folium.LayerControl().add_to(m)

# Adicionar a legenda da paleta de cores
m.add_child(colormap)

# Salvar o mapa em um arquivo HTML
m.save('mapa_velocidade_internet.html')

# Exibir o mapa
m